In [2]:
import pandas as pd
import pickle
import numpy as np

In [3]:
go_terms_unprocessed = pd.read_csv('/Users/michael/Data/Junk_data/mart_export (1).txt')


In [4]:
go_terms_unprocessed = go_terms_unprocessed.rename(columns = {'Gene stable ID':'GeneID',})
go_terms_unprocessed = go_terms_unprocessed.drop(columns = ['Transcript stable ID','GO domain','GO term evidence code',])
go_terms_unprocessed

,GeneID,GO term name,GO term definition,GO term accession,GOSlim GOA Accession(s),GOSlim GOA Description,TAIR ID
0,AT5G16970,oxidoreductase activity,Catalysis of an oxidation-reduction (redox) re...,GO:0016491,GO:0006950,response to stress,AT5G16970
1,AT5G16970,response to oxidative stress,Any process that results in a change in state ...,GO:0006979,GO:0006950,response to stress,AT5G16970
2,AT5G16970,"oxidoreductase activity, acting on the CH-CH g...",Catalysis of an oxidation-reduction (redox) re...,GO:0016628,GO:0006950,response to stress,AT5G16970
3,AT2G43120,nucleus,A membrane-bounded organelle of eukaryotic cel...,GO:0005634,GO:0005634,nucleus,AT2G43120
4,AT2G43120,metal ion binding,Binding to a metal ion.,GO:0046872,GO:0003674,molecular_function,AT2G43120
...,...,...,...,...,...,...,...
1223719,AT1G18745,nucleolus,"A small, dense body one or more of which are p...",GO:0005730,GO:0005730,nucleolus,AT1G18745
1223720,AT4G36648,translation,The cellular metabolic process in which a prot...,GO:0006412,GO:0006412,translation,AT4G36648
1223721,AT4G36648,triplet codon-amino acid adaptor activity,The codon binding activity of a tRNA that posi...,GO:0030533,GO:0006412,translation,AT4G36648
1223722,AT4G36648,translation,The cellular metabolic process in which a prot...,GO:0006412,GO:0003674,molecular_function,AT4G36648


In [5]:
unusable_samples = [
    "A2450525897_n01_LICRNA01_A06",
    "A2450525897_n01_LICRNA01_A08",
    "A2450525897_n01_LICRNA01_C11",
    "A2450525897_n01_LICRNA01_D11",
    "A2450525897_n01_LICRNA01_D12",
    "A2449446903_n01_LICRNA02_F01",
    "B2449500127_n01_LICRNA04_A04",
    "B2449500127_n01_LICRNA04_A06",
    "B2449500127_n01_LICRNA04_A07",

]
borderline_unusable = ['A2450525897_n01_LICRNA01_F03', 'A2450525897_n01_LICRNA01_G07','A2449446903_n01_LICRNA02_A04','A2449446903_n01_LICRNA02_H10',"B2449500127_n01_LICRNA04_H02","B2449500127_n01_LICRNA04_H11"]
all_unsable = borderline_unusable+unusable_samples

In [6]:
metadata = pd.read_csv('/Users/michael/Data/Luke_terrace_experiment/General_data/Plates_1_to_4_metadata_merged_luke.csv', index_col = 0,)
metadata = metadata.drop(columns= ['arb.sort','sample-id','Ambiguous Unstranded','Ambiguous Forward','Multimapping','Unmapped Over Mapped'])
metadata['Date and Time'] = metadata['date'] +' ' +  metadata['time']
luke_time_data_format = "%-m/%-d/%y %-H:%-M"
metadata['Date and Time'] = pd.to_datetime(metadata['Date and Time'], format = luke_time_data_format)

In [7]:
transcriptome = pd.read_csv('/Users/michael/Data/Luke_terrace_experiment/General_data/plate1_4_tpm_normalized.csv', index_col = 0)
rows_to_drop_expression_data = ['A2450525897_n01_undetermined',
                'A2449446903_n01_undetermined',
                'B250508004_n01_undetermined',
                'B2449500127_n01_undetermined']
transcriptome = transcriptome.drop(index = rows_to_drop_expression_data)
transcriptome = transcriptome.sort_index()
trimmed_transcriptome = transcriptome.drop(index = all_unsable)
trimmed_metadata = metadata.drop(index= all_unsable)


In [8]:
go_terms_trimmed = go_terms_unprocessed.loc[go_terms_unprocessed['GeneID'].isin(transcriptome.T.index)]


In [9]:
go_terms_trimmed ['Present'] = 1
go_terms_clean = go_terms_trimmed[['GeneID','GO term accession','Present']]
go_terms_clean = go_terms_clean.drop_duplicates()

matrixed_version = go_terms_clean.pivot(index='GeneID',
                                        columns='GO term accession',
                                        values='Present')



filled_matrixed_version = matrixed_version.fillna(0)
filled_matrixed_version

/var/folders/nk/6xkk9sgn1pz4ff1b36sfq3y40000gt/T/ipykernel_59066/3147939452.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  go_terms_trimmed ['Present'] = 1


GO term accession,GO:0000002,GO:0000009,GO:0000012,GO:0000014,GO:0000015,GO:0000023,GO:0000027,GO:0000028,GO:0000030,GO:0000033,...,GO:2001011,GO:2001057,GO:2001070,GO:2001142,GO:2001143,GO:2001243,GO:2001280,GO:2001289,GO:2001294,GO:2001295
GeneID,,,,,,,,,,,,,,,,,,,,,
AT1G01010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AT1G01020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AT1G01030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AT1G01040,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AT1G01050,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AT5G67590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AT5G67600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AT5G67610,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
def generate_contigency_tables(go_annotations,gene_list, use_background_genes = False, background_genes =[]):
    """Takes list of Go Annotations as a binary matrix with Genes as the index and GO terms as columns, and a list of genes, and 
    generates the contigency tables on which to run a Fisher Exact Test for enrichment. 
    Args:
        go_annotations (Panda DataFrame): Binary Matrix, with genes as index and GO Terms as colummns. Generated by Go_annotations function (may not
        work for non plant species idk check the code for it its been a while)
        gene_list (list): What is says on the tin
    """

    if use_background_genes == True:
        go_annotations = go_annotations.loc[go_annotations.columns.intersection(background_genes)]
    gene_list = list(set(go_annotations.index) & set(gene_list))
    background_distribution_num_genes_annotated_with_go_term = go_annotations.sum(axis =0)
    background_dipstribution_num_genes_that_lack_go_term = go_annotations.shape[0]-background_distribution_num_genes_annotated_with_go_term
    annotation_subset_to_genes_in_list = go_annotations.loc[gene_list]
    go_subset_distribution_num_genes_annotatated_with_go_term = annotation_subset_to_genes_in_list.sum(axis =0)
    go_subset_distribution_num_genes_that_lack_go_term = annotation_subset_to_genes_in_list.shape[0] - go_subset_distribution_num_genes_annotatated_with_go_term
    
    all_go_contingency_tables = []
    i = 0
    for term_of_go in background_distribution_num_genes_annotated_with_go_term:
        contig_table = [[(term_of_go - go_subset_distribution_num_genes_annotatated_with_go_term[i]),go_subset_distribution_num_genes_annotatated_with_go_term[i]],[(background_dipstribution_num_genes_that_lack_go_term[i]-go_subset_distribution_num_genes_that_lack_go_term[i]),go_subset_distribution_num_genes_that_lack_go_term[i]]]
        all_go_contingency_tables.append(contig_table)
        i +=1
    return all_go_contingency_tables


def fishers_exact_on_contigency_tables(all_go_contigency_tables,original_GO_term_panda, return_odds_ratio = False):
    """Takes Table from generate_contigency_tables and runs fishers exact on all the tables, without correcting the P-values. Returns
    either just a Panda with the p values, or a Panda of lists for odds ratios, P values, just like stats.fisher_exact. If you're wondering if 
    you want the Odds Ratios, you don't want them

    Args:
        all_go_contigency_tables (list): Contigency Tables formatted as list of lists from generate_contigency_tables
        return_odds_ratio (bool): Boolean for if you want it to return both the odds ratios and the P values or just the P Values. Defaults to just the 
        P Values
        original_GO_term_panda (Panda DataFrame):  Binary Matrix, with genes as index and GO Terms as colummns. Generated by Go_annotations function (may not
        work for non plant species idk check the code for it its been a while)  Same as above
    """
    import scipy.stats as stats
    import pandas as pd

    list_of_p_values_for_go_terms = []
    list_of_odds_ratios_for_go_terms = []

    for table in all_go_contigency_tables:
        odds, p_value = stats.fisher_exact(table = table, alternative = 'less')
        list_of_p_values_for_go_terms.append(p_value)
        list_of_odds_ratios_for_go_terms.append(odds)

    if return_odds_ratio == True:

        go_terms_with_P_value_and_odds = pd.DataFrame(data = list(zip(list_of_p_values_for_go_terms, list_of_odds_ratios_for_go_terms)),index = original_GO_term_panda.columns, columns = ['P_value', 'Odds Ratios'])
        return go_terms_with_P_value_and_odds

    if return_odds_ratio == False:

        go_terms_with_P_value = pd.DataFrame(data = list_of_p_values_for_go_terms,index = original_GO_term_panda.columns, columns = ['P_value'])
        return go_terms_with_P_value

        
def multi_hypothesis_correct_fishers_exact(go_terms_with_P_value, alpha = 0.05, method = 'indep', is_sorted = False, panda_has_odds_ratio = False, Include_rejected_boolean_array = False):
    """Runs Multiple Hypothesis Testing via statsmodels.stats.multitest.fdrcorrection. Takes all values for that, as well as Panda of go terms from previous step

    Args:
        go_terms_with_P_value (Panda Dataframe): Pandas Dataframe from fishers_exact_on_contigency_tables
        alpha (float): Familywise error rate, defaults to 0.05
        method (str): Which method to use for FDR correction. {'i', 'indep', 'p', 'poscorr'} all refer to fdr_bh (Benjamini/Hochberg for independent or positively correlated tests).
         {'n', 'negcorr'} both refer to fdr_by (Benjamini/Yekutieli for general or negatively correlated tests). Defaults to 'indep'.
        is_sorted (bool): _f False (default), the p_values will be sorted, but the corrected pvalues are in the original order. If True, then it assumed that the pvalues are already sorted in ascending order.
        panda_has_odds_ratio (bool): Did you ignore my previous Docstring in fishers_exact_on_contigency_tables and add the odds ratio? If so this fixes 
        your foolishness and drops that column
        Include_rejected_boolean_array (bool): Makes function return a tuple of a boolean array of if a hypothesis was rejected, and the normal Panda of corrected_p_values. Don't do this just use your eyes
    """
    import statsmodels.stats.multitest   
    import pandas as pd

    if panda_has_odds_ratio == True:
        go_terms_with_P_value = go_terms_with_P_value[['P_value']]
    
    rejected, corrected_pvalues = statsmodels.stats.multitest.fdrcorrection(pvals = go_terms_with_P_value['P_value'].to_list(), alpha = alpha, method = method, is_sorted= is_sorted)
    corrected_pvalues = corrected_pvalues.tolist()
    go_enrichment_results = pd.DataFrame(corrected_pvalues,index = go_terms_with_P_value.index,columns=['P_value'])
    
    if Include_rejected_boolean_array == True:
        return rejected, go_enrichment_results
    
    if Include_rejected_boolean_array == False:
        return go_enrichment_results

def test_using_kwargs(go_terms_with_P_value,panda_has_odds_ratio = False, Include_rejected_boolean_array = False, **kwargs):
    """Runs Multiple Hypothesis Testing via statsmodels.stats.multitest.fdrcorrection. Takes all values for that, as well as Panda of go terms from previous step

    Args:
        go_terms_with_P_value (Panda Dataframe): Pandas Dataframe from fishers_exact_on_contigency_tables
        kwargs : Refer to statsmodels.stats.multitest.fdrcorrection for kwargs
        alpha (float): Familywise error rate, defaults to 0.05
        method (str): Which method to use for FDR correction. {'i', 'indep', 'p', 'poscorr'} all refer to fdr_bh (Benjamini/Hochberg for independent or positively correlated tests).
         {'n', 'negcorr'} both refer to fdr_by (Benjamini/Yekutieli for general or negatively correlated tests). Defaults to 'indep'.
        is_sorted (bool): _f False (default), the p_values will be sorted, but the corrected pvalues are in the original order. If True, then it assumed that the pvalues are already sorted in ascending order.
        panda_has_odds_ratio (bool): Did you ignore my previous Docstring in fishers_exact_on_contigency_tables and add the odds ratio? If so this fixes 
        your foolishness and drops that column
        Include_rejected_boolean_array (bool): Makes function return a tuple of a boolean array of if a hypothesis was rejected, and the normal Panda of corrected_p_values. Don't do this just use your eyes
    """
    import statsmodels.stats.multitest   
    import pandas as pd

    if panda_has_odds_ratio == True:
        go_terms_with_P_value = go_terms_with_P_value[['P_value']]
    
    rejected, corrected_pvalues = statsmodels.stats.multitest.fdrcorrection(pvals = go_terms_with_P_value['P_value'].to_list(), **kwargs)
    go_enrichment_results = pd.DataFrame(corrected_pvalues,index = go_terms_with_P_value.index,columns=['P_value'])
    
    if Include_rejected_boolean_array == True:
        return rejected, go_enrichment_results
    
    if Include_rejected_boolean_array == False:
        return go_enrichment_results

In [11]:
long_term_transcriptome = trimmed_transcriptome.loc[trimmed_metadata['Experiment Type'] == 'Long Term']
long_term_metadata = trimmed_metadata.loc[trimmed_metadata['Experiment Type'] == 'Long Term']

groupby_timepoint = long_term_transcriptome.groupby(long_term_metadata['timepoint']).mean()
short_transcriptome = groupby_timepoint.loc[:, (groupby_timepoint  > 1).mean() > .5]

In [12]:
dbfile_2 = open('/Users/michael/Data/Junk_data/gene_spearman_correlation_with_bacteria.p', 'rb')    
full_spearman_correlation_matrix = pickle.load(dbfile_2)

In [13]:
full_spearman_correlation_matrix

,AT1G01010,AT1G01020,AT1G01030,AT1G01040,AT1G01050,AT1G01060,AT1G01070,AT1G01080,AT1G01090,AT1G01100,...,ArthCt115,ArthCr091,ArthCr090,ArthCr089,ArthCt114,ArthCt124,ArthCr088,ArthCt112,ArthCp088,ArthCp085
Genus,,,,,,,,,,,,,,,,,,,,,
Actinomycetospora,-0.037245,0.359304,0.215254,0.235246,0.114200,0.003286,-0.002191,0.130631,0.230590,0.135835,...,0.036706,-0.097768,0.170889,0.095577,0.131179,-0.125428,-0.003286,0.156532,-0.035054,-0.252499
Actinoplanes,-0.009582,0.145106,0.005476,-0.004107,0.088706,-0.125394,0.113895,-0.088159,-0.040246,-0.098015,...,-0.140437,0.078303,-0.158248,-0.158248,0.205886,-0.073648,-0.223682,-0.026972,-0.015880,-0.301164
Aeromicrobium,0.065863,0.026906,-0.052691,0.223375,-0.054372,0.034753,0.161155,0.287557,0.197590,0.440024,...,-0.174832,-0.371358,-0.019058,0.158353,0.068106,-0.177691,0.044283,-0.233777,-0.132287,-0.365192
Aetherobacter,-0.244510,-0.247898,-0.351801,-0.073692,-0.308037,-0.011858,0.066916,-0.609016,-0.611840,-0.662379,...,0.180109,0.345307,0.479703,0.476315,0.264274,0.387376,0.432834,0.389971,0.029364,0.239146
Agrobacterium,0.209748,0.263417,-0.050931,-0.096112,0.039704,0.314075,0.283954,-0.158817,-0.076397,-0.107338,...,-0.166947,0.105422,-0.040526,0.014513,-0.044359,0.004107,0.087623,0.378064,0.135542,0.220427
Amnibacterium_382409,0.042205,0.203093,0.320076,-0.048153,0.375877,0.301948,0.101121,0.755436,0.744389,0.511555,...,-0.306858,-0.491161,-0.612960,-0.580103,-0.426013,-0.684340,-0.478981,-0.279609,0.014729,0.002549
Aquabacterium_A_592457,0.099716,-0.018531,0.142662,-0.066183,0.044122,-0.195314,-0.205904,0.315033,0.183843,0.280323,...,0.204536,0.134720,-0.143838,-0.158252,0.180313,-0.086774,-0.108247,-0.140181,-0.492698,-0.480638
Aureimonas_A_501549,-0.163975,-0.291541,-0.025459,-0.212702,0.174651,0.069532,-0.116343,-0.080482,-0.050096,-0.103203,...,0.424012,-0.094717,0.160690,-0.063509,0.149466,0.002737,0.060224,-0.016975,0.096907,0.070627
Blastococcus,0.254177,0.242126,0.131471,0.373596,0.401260,0.371131,0.169543,0.493016,0.603944,0.253355,...,-0.242346,-0.327582,-0.376609,-0.209532,-0.591893,-0.696522,-0.066831,-0.170936,-0.227335,-0.012051


In [14]:
top_200_spearman = np.array([full_spearman_correlation_matrix.T[c].nlargest(200).index.values for c in full_spearman_correlation_matrix.T])

In [15]:
df_top200 = pd.DataFrame(index = full_spearman_correlation_matrix.index, data = top_200_spearman)
df_top200

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
Genus,,,,,,,,,,,,,,,,,,,,,
Actinomycetospora,AT4G28020,AT5G47610,AT1G05560,AT2G37050,AT1G48620,ArthCt091,AT5G03415,AT4G12970,AT3G54220,AT5G02010,...,AT3G26690,AT3G02640,AT3G18390,AT5G53486,AT5G03555,AT5G12230,AT5G23430,AT2G32415,AT3G04485,AT4G37550
Actinoplanes,AT2G38995,AT1G01900,AT1G70890,AT5G17700,AT1G69523,AT3G62750,AT3G48185,AT4G22635,AT5G01495,AT4G33470,...,AT3G16280,AT1G80130,AT2G22970,AT5G07990,AT1G69572,AT2G47880,AT5G52880,AT1G01250,AT1G06420,AT3G04310
Aeromicrobium,AT2G04430,AT1G12440,AT2G38860,AT1G35260,AT1G08213,AT3G09150,AT3G02060,AT1G13145,AT2G37130,AT3G11591,...,AT1G61870,AT2G32020,AT1G03110,AT1G48160,AT2G31660,AT5G60960,AT5G55740,AT5G58130,AT3G48870,AT3G51450
Aetherobacter,ArthCp002,ArthCp030,AT1G28010,AT5G24150,ArthCp007,ArthCp053,AT3G28500,ArthCp017,ArthCp015,ArthCp042,...,AT5G54650,AT2G36590,AT5G36920,AT5G58770,AT5G45190,AT1G52565,AT1G69750,AT3G27770,AT1G21120,ArthCr088
Agrobacterium,AT5G46500,AT3G45443,AT3G12110,AT1G02960,AT2G32250,AT1G16820,AT2G10606,AT5G48340,AT2G40670,AT4G03845,...,AT1G05780,AT2G04032,AT3G51150,AT1G63800,AT2G17710,AT3G44444,AT1G21250,AT1G57980,AT1G67865,AT5G26200
Amnibacterium_382409,AT2G22230,AT5G16390,AT1G53520,AT5G52920,AT1G73620,AT5G19370,AT1G33170,AT4G35335,AT3G18080,AT3G51820,...,AT1G07780,AT4G16155,AT5G35100,AT2G41770,AT5G13030,AT5G59970,AT2G43360,AT3G06430,AT5G39830,AT3G46740
Aquabacterium_A_592457,AT3G46630,AT5G08415,AT2G42710,AT4G29440,AT2G43290,AT2G24970,AT3G13445,AT1G52270,AT5G03370,AT2G39900,...,AT2G25830,AT2G29310,AT5G52040,AT5G55050,AT2G19470,AT2G25840,AT1G08610,AT1G52860,AT3G13570,AT4G23540
Aureimonas_A_501549,AT3G21670,AT2G27420,AT5G42760,AT3G24460,AT1G64780,AT3G27170,AT4G27360,AT1G19490,AT3G51375,AT3G47430,...,AT4G14770,AT5G21900,AT2G09080,AT1G14150,AT3G55580,AT4G25290,AT3G52130,AT1G18810,AT2G20724,AT4G28290
Blastococcus,AT2G21410,AT5G05620,AT4G15800,AT3G50800,AT5G17520,AT3G46550,AT1G29890,AT3G09035,AT3G06550,AT5G22050,...,AT2G32295,AT4G22580,AT5G05830,AT3G25530,AT4G39050,AT3G59920,AT4G27800,AT1G76390,AT3G56360,AT5G59290


In [16]:
df_top200.to_csv('/Users/michael/Data/Luke_terrace_experiment/Output_for_Luke/top_200_spearman_correlated_genes_per_bacteria.csv',)

In [115]:
top_200_spearman[0]

array(['AT4G28020', 'AT5G47610', 'AT1G05560', 'AT2G37050', 'AT1G48620',
       'ArthCt091', 'AT5G03415', 'AT4G12970', 'AT3G54220', 'AT5G02010',
       'AT5G60190', 'AT5G53900', 'AT2G33310', 'AT5G56460', 'AT5G53440',
       'AT5G57130', 'AT1G17510', 'AT1G77480', 'AT1G04210', 'AT1G05720',
       'AT4G15840', 'AT1G60200', 'AT5G23580', 'AT2G19080', 'AT4G39235',
       'AT5G01370', 'AT4G31160', 'AT1G70180', 'AT1G13640', 'AT5G04290',
       'AT5G14460', 'AT2G23520', 'AT2G30140', 'AT1G54490', 'AT3G11580',
       'AT1G42550', 'AT3G54190', 'AT5G43180', 'AT4G37020', 'ArthCp006',
       'AT4G24660', 'AT3G14160', 'AT3G10130', 'AT5G46840', 'AT1G32240',
       'AT2G34070', 'AT3G04600', 'AT5G12900', 'AT3G48800', 'AT3G20930',
       'AT1G64450', 'AT1G47970', 'AT3G21690', 'AT3G48150', 'AT5G57390',
       'AT1G53050', 'AT3G10050', 'AT5G53800', 'AT2G42080', 'AT1G31935',
       'AT1G68720', 'AT1G32370', 'AT2G30170', 'AT1G06110', 'AT4G21705',
       'AT2G08650', 'AT5G47690', 'AT4G00720', 'AT2G47350', 'AT2G

In [116]:
short_transcriptome.T.index

Index(['AT1G01010', 'AT1G01020', 'AT1G01030', 'AT1G01040', 'AT1G01050',
       'AT1G01060', 'AT1G01070', 'AT1G01080', 'AT1G01090', 'AT1G01100',
       ...
       'ArthCt115', 'ArthCr091', 'ArthCr090', 'ArthCr089', 'ArthCt114',
       'ArthCt124', 'ArthCr088', 'ArthCt112', 'ArthCp088', 'ArthCp085'],
      dtype='object', length=15091)

In [127]:
#bacteria_one = generate_contigency_tables(go_annotations=filled_matrixed_version, gene_list=top_200_spearman[1], use_background_genes = True, background_genes = short_transcriptome.T.index)
bacteria_one = generate_contigency_tables(go_annotations=filled_matrixed_version, gene_list=df_top200.loc['Chryseobacterium_A_796612'].to_list(),)


/var/folders/nk/6xkk9sgn1pz4ff1b36sfq3y40000gt/T/ipykernel_51136/3959193131.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  contig_table = [[(term_of_go - go_subset_distribution_num_genes_annotatated_with_go_term[i]),go_subset_distribution_num_genes_annotatated_with_go_term[i]],[(background_dipstribution_num_genes_that_lack_go_term[i]-go_subset_distribution_num_genes_that_lack_go_term[i]),go_subset_distribution_num_genes_that_lack_go_term[i]]]


In [128]:
bacteria_one_fishers_results = fishers_exact_on_contigency_tables(bacteria_one, filled_matrixed_version)


In [129]:
corrected_bacteria_one_reults = multi_hypothesis_correct_fishers_exact(bacteria_one_fishers_results)

In [130]:
bacteria_one_fishers_results.sort_values(by = 'P_value', ascending = True)

,P_value
GO term accession,
GO:0003735,3.798732e-32
GO:0005840,3.174077e-29
GO:1990904,1.673409e-27
GO:0006412,1.673510e-21
GO:0019843,1.086118e-10
...,...
GO:0000070,1.000000e+00
GO:0000064,1.000000e+00
GO:0000062,1.000000e+00


In [131]:
corrected_bacteria_one_reults.sort_values(by = 'P_value', ascending = True).head(50)

,P_value
GO term accession,
GO:0003735,1.963945e-28
GO:0005840,8.204990e-26
GO:1990904,2.883841e-24
GO:0006412,2.163012e-18
GO:0019843,1.123046e-07
GO:0009507,7.290481e-07
GO:0015935,1.628507e-06
GO:0003723,9.768336e-06
GO:0009536,5.333727e-05


In [17]:
dbfile_1 = open('/Users/michael/Data/Junk_data/gene_pearson_correlation_with_bacteria.p', 'rb')    
full_correlation_matrix = pickle.load(dbfile_1)

In [18]:
top_200_pearson = np.array([full_correlation_matrix.T[c].nlargest(200).index.values for c in full_correlation_matrix.T])

In [20]:
df_top200_pearson = pd.DataFrame(index = full_correlation_matrix.index, data = top_200_pearson)
df_top200

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
Genus,,,,,,,,,,,,,,,,,,,,,
Actinomycetospora,AT4G28020,AT5G47610,AT1G05560,AT2G37050,AT1G48620,ArthCt091,AT5G03415,AT4G12970,AT3G54220,AT5G02010,...,AT3G26690,AT3G02640,AT3G18390,AT5G53486,AT5G03555,AT5G12230,AT5G23430,AT2G32415,AT3G04485,AT4G37550
Actinoplanes,AT2G38995,AT1G01900,AT1G70890,AT5G17700,AT1G69523,AT3G62750,AT3G48185,AT4G22635,AT5G01495,AT4G33470,...,AT3G16280,AT1G80130,AT2G22970,AT5G07990,AT1G69572,AT2G47880,AT5G52880,AT1G01250,AT1G06420,AT3G04310
Aeromicrobium,AT2G04430,AT1G12440,AT2G38860,AT1G35260,AT1G08213,AT3G09150,AT3G02060,AT1G13145,AT2G37130,AT3G11591,...,AT1G61870,AT2G32020,AT1G03110,AT1G48160,AT2G31660,AT5G60960,AT5G55740,AT5G58130,AT3G48870,AT3G51450
Aetherobacter,ArthCp002,ArthCp030,AT1G28010,AT5G24150,ArthCp007,ArthCp053,AT3G28500,ArthCp017,ArthCp015,ArthCp042,...,AT5G54650,AT2G36590,AT5G36920,AT5G58770,AT5G45190,AT1G52565,AT1G69750,AT3G27770,AT1G21120,ArthCr088
Agrobacterium,AT5G46500,AT3G45443,AT3G12110,AT1G02960,AT2G32250,AT1G16820,AT2G10606,AT5G48340,AT2G40670,AT4G03845,...,AT1G05780,AT2G04032,AT3G51150,AT1G63800,AT2G17710,AT3G44444,AT1G21250,AT1G57980,AT1G67865,AT5G26200
Amnibacterium_382409,AT2G22230,AT5G16390,AT1G53520,AT5G52920,AT1G73620,AT5G19370,AT1G33170,AT4G35335,AT3G18080,AT3G51820,...,AT1G07780,AT4G16155,AT5G35100,AT2G41770,AT5G13030,AT5G59970,AT2G43360,AT3G06430,AT5G39830,AT3G46740
Aquabacterium_A_592457,AT3G46630,AT5G08415,AT2G42710,AT4G29440,AT2G43290,AT2G24970,AT3G13445,AT1G52270,AT5G03370,AT2G39900,...,AT2G25830,AT2G29310,AT5G52040,AT5G55050,AT2G19470,AT2G25840,AT1G08610,AT1G52860,AT3G13570,AT4G23540
Aureimonas_A_501549,AT3G21670,AT2G27420,AT5G42760,AT3G24460,AT1G64780,AT3G27170,AT4G27360,AT1G19490,AT3G51375,AT3G47430,...,AT4G14770,AT5G21900,AT2G09080,AT1G14150,AT3G55580,AT4G25290,AT3G52130,AT1G18810,AT2G20724,AT4G28290
Blastococcus,AT2G21410,AT5G05620,AT4G15800,AT3G50800,AT5G17520,AT3G46550,AT1G29890,AT3G09035,AT3G06550,AT5G22050,...,AT2G32295,AT4G22580,AT5G05830,AT3G25530,AT4G39050,AT3G59920,AT4G27800,AT1G76390,AT3G56360,AT5G59290


In [21]:
df_top200_pearson.to_csv('/Users/michael/Data/Luke_terrace_experiment/Output_for_Luke/top_200_pearson_correlated_genes_per_bacteria.csv',)